In [27]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import plotly as ply
import os
import re
import requests
import json
from IPython.core.display import HTML

# Load Shot Data

In [42]:
shot_data = pd.read_csv('data/fantasy-league/shot_data.csv', index_col=0)
shot_data['Timestamp'] = pd.to_datetime(shot_data['Timestamp'], format='%Y-%m-%d') 

In [43]:
shot_data.head()

,Timestamp,Score,Player Advantage,Minute,Player,Squad,Against,Outcome,Distance,Body Part,Notes,SCA 1 Player,SCA 1 Event,SCA 2 Player,SCA 2 Event,Threat
0,2016-08-13 12:30:00,0.0,0.0,147,Riyad Mahrez,Leicester City,Hull City,Goal,NaN,NaN,Penalty Kick,Goal,NaN,NaN,NaN,720.0
1,2016-08-13 17:30:00,1.0,0.0,4,Sergio Agüero,Manchester City,Sunderland,Goal,NaN,NaN,Penalty Kick,—,NaN,NaN,NaN,720.0
2,2016-08-15 20:00:00,1.0,0.0,147,Eden Hazard,Chelsea,West Ham United,Goal,NaN,NaN,Penalty Kick,Yellow Card,NaN,NaN,NaN,627.0
3,2016-08-19 20:00:00,2.0,0.0,152,Zlatan Ibrahimović,Manchester United,Southampton,Goal,NaN,NaN,Penalty Kick,Goal,NaN,NaN,NaN,627.0
4,2016-08-20 12:30:00,1.0,0.0,27,Sergio Agüero,Manchester City,Stoke City,Goal,NaN,NaN,Penalty Kick,Yellow Card,NaN,NaN,NaN,NaN


In [44]:
teams = {'Arsenal':'Arsenal',
 'Aston Villa':'Aston Villa',
 'Bournemouth':'Bournemouth',
 'Brighton':'Brighton',
 'Brighton & Hove Albion':'Brighton',
 'Burnley':'Burnley',
 'Cardiff City':'Cardiff',
 'Chelsea':'Chelsea',
 'Crystal Palace':'Crystal Palace',
 'Everton':'Everton',
 'Fulham':'Fulham',
 'Huddersfield':'Huddersfield',
 'Huddersfield Town':'Huddersfield',
 'Hull City':'Hull',
 'Leeds United':'Leeds',
 'Leicester City':'Leicester',
 'Liverpool':'Liverpool',
 'Manchester City':'Man City',
 'Manchester United':'Man United',
 'Manchester Utd':'Man United',
 'Middlesbrough':'Middlesbrough',
 'Newcastle United':'Newcastle',
 'Newcastle Utd':'Newcastle',
 'Norwich City':'Norwich',
 'Sheffield United':'Sheffield United',
 'Sheffield Utd':'Sheffield United',
 'Southampton':'Southampton',
 'Stoke City':'Stoke',
 'Sunderland':'Sunderland',
 'Swansea City':'Swansea',
 'Tottenham':'Tottenham',
 'Tottenham Hotspur':'Tottenham',
 'Watford':'Watford',
 'West Brom':'West Brom',
 'West Bromwich Albion':'West Brom',
 'West Ham':'West Ham',
 'West Ham United':'West Ham',
 'Wolverhampton Wanderers':'Wolves',
 'Wolves':'Wolves'}

In [45]:
shot_data['Squad'] = [teams[s] for s in shot_data['Squad']]

# Load Non-shot Data

In [46]:
non_shot_data = pd.read_csv('data/non-shot-xG/non_shot_data.csv', index_col=0)
non_shot_data['Date'] = pd.to_datetime(non_shot_data['Date'], format='%d/%m/%Y')

In [47]:
non_shot_data.head()

,Date,HomeTeam,AwayTeam,Referee,HC,AC,HF,AF,HY,AY,HR,AR,FTHG,FTAG,FTR
GameID,,,,,,,,,,,,,,,
1,2016-08-13,Burnley,Swansea,J Moss,7,4,10,14,3,2,0,0,0,1,A
2,2016-08-13,Crystal Palace,West Brom,C Pawson,3,6,12,15,2,2,0,0,0,1,A
3,2016-08-13,Everton,Tottenham,M Atkinson,5,6,10,14,0,0,0,0,1,1,D
4,2016-08-13,Hull,Leicester,M Dean,5,3,8,17,2,2,0,0,2,1,H
5,2016-08-13,Man City,Sunderland,R Madley,9,6,11,14,1,2,0,0,2,1,H


# Match Data

In [48]:
# shot_data = shot_data.loc[shot_data['Timestamp'] <= non_shot_data['Date'].max()]

In [49]:
non_shot_data['Date'] = [d.date() for d in non_shot_data['Date']]

In [50]:
arr = np.asarray([d for d in non_shot_data['Date']]) == pd.to_datetime('2018-01-12').date()

In [51]:
non_shot_data_groups = non_shot_data.groupby('Date').groups

In [52]:
shot_data['MatchID'] = np.NaN

## Add MatchID

In [53]:
errors = []
try:
    for i, s in shot_data.iterrows():
        date = s['Timestamp'].date()
        if date in non_shot_data_groups:
            indices = non_shot_data_groups[date]
            group = non_shot_data.loc[indices]
            home = group['HomeTeam'] == s['Squad']
            if sum(home) > 0:
                shot_data.loc[i, 'MatchID'] = group.loc[home].index[0]
                continue
            away = group['AwayTeam'] == s['Squad']
            if sum(away) > 0:
                shot_data.loc[i, 'MatchID'] = group.loc[away].index[0]
                continue

        else:
            errors.append(i)
except:
    print(i)

In [61]:
 shot_data = shot_data[~pd.isnull(shot_data['MatchID'])]

In [62]:
path = os.path.join(os.getcwd(), "output/shot_data.csv")
shot_data.to_csv(path, index=False)